## Import Match Data

In [44]:
import pandas as pd
import numpy as np
import sqlite3

In [45]:
def import_match_data():
    import pandas as pd
    import sqlite3
    con_matches = sqlite3.connect('database.sqlite') #connect to matches DB
    c = con_matches.cursor() #connect cursor
    df = pd.read_sql("""
        SELECT *
        FROM(
            SELECT *
            FROM(
                SELECT 
                    Match_ID, 
                    "Home" as Match_Team, 
                    Div, 
                    Season, 
                    Date, 
                    HomeTeam as Team, 
                    AwayTeam as Opponent, 
                    (FTHG - FTAG) as Result
                FROM Matches
                WHERE Season = 2011
                )
            UNION ALL
            SELECT *
            FROM(
                SELECT 
                    Match_ID, 
                    "Away" as Match_Team, 
                    Div, 
                    Season, 
                    Date, 
                    AwayTeam as Team, 
                    HomeTeam as Opponent, 
                    -(FTHG - FTAG) as Result
                FROM Matches
                WHERE Season = 2011
                )
            )
        ORDER BY Match_ID, Match_Team Desc;
    """, con_matches)
    df.Date = pd.to_datetime(df.Date) # convert Date to datetime format
    
    return df.copy()

## Various Testing

In [46]:
team_wins = import_match_data()

In [47]:
team_wins.head()

,Match_ID,Match_Team,Div,Season,Date,Team,Opponent,Result
0,1092,Home,D1,2011,2012-03-31,Nurnberg,Bayern Munich,-1
1,1092,Away,D1,2011,2012-03-31,Bayern Munich,Nurnberg,1
2,1093,Home,D1,2011,2011-12-11,Stuttgart,Bayern Munich,-1
3,1093,Away,D1,2011,2011-12-11,Bayern Munich,Stuttgart,1
4,1094,Home,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,-1


In [48]:
team_wins[0:5].Date.astype(np.int64) // 10**9

0    1333152000
1    1333152000
2    1323561600
3    1323561600
4    1313193600
Name: Date, dtype: int64

In [49]:
team_wins.head()

,Match_ID,Match_Team,Div,Season,Date,Team,Opponent,Result
0,1092,Home,D1,2011,2012-03-31,Nurnberg,Bayern Munich,-1
1,1092,Away,D1,2011,2012-03-31,Bayern Munich,Nurnberg,1
2,1093,Home,D1,2011,2011-12-11,Stuttgart,Bayern Munich,-1
3,1093,Away,D1,2011,2011-12-11,Bayern Munich,Stuttgart,1
4,1094,Home,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,-1


## Import Rain Data

### Get data using requests

In [50]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [51]:
def dark_sky_url(time):
    """ Unix time preffered """
    import dark_sky_key
    key = dark_sky_key.import_dark_sky_key()
    lat = str(52.52437)
    lon = str(13.41053)
    excl = 'exclude=currently,minutely,hourly,alerts,flags'
    url = 'https://api.darksky.net/forecast/{}/{},{},{}?{}'.format(key,lat,lon,time,excl)
    return url

### Get data

In [119]:
def get_dark_sky_date(time):
    """ Data returned as a json conent """
    url = dark_sky_url(time)
    req = requests.get(url)
    content = json.loads(req.content)
    return content

### Parse json and Return Whether Rainy

In [134]:
def check_rain(time):
    content = get_dark_sky_date(time)
    try:
        max_precip = content['daily']['data'][0]['precipIntensityMax']
        # NOTE: any precipitation is considered 'rain' or similar
        if max_precip > 0:
            rain = 1
        else:
            rain = 0
        return rain
    except:
        rain = 0
        return rain

In [132]:
time = 1333152000
check_rain(time)

1

In [115]:
    url = dark_sky_url(time)
    req = requests.get(url)

In [116]:
type(req)

requests.models.Response

### Testing data

In [133]:
# Content used for testing

content = {'latitude': 52.52437,
     'longitude': 13.41053,
     'timezone': 'Europe/Berlin',
     'daily': {'data': [{'time': 1333144800,
    'summary': 'Possible drizzle in the morning and afternoon.',
    'icon': 'rain',
    'sunriseTime': 1333168980,
    'sunsetTime': 1333215600,
    'moonPhase': 0.28,
    'precipIntensity': 0.0003,
    'precipIntensityMax': 0.0029,
    'precipIntensityMaxTime': 1333180800,
    'precipProbability': 0.56,
    'precipType': 'rain',
    'temperatureHigh': 44.59,
    'temperatureHighTime': 1333188000,
    'temperatureLow': 30.09,
    'temperatureLowTime': 1333256400,
    'apparentTemperatureHigh': 36.29,
    'apparentTemperatureHighTime': 1333188000,
    'apparentTemperatureLow': 23.95,
    'apparentTemperatureLowTime': 1333256400,
    'dewPoint': 33.23,
    'humidity': 0.7,
    'windSpeed': 16.7,
    'windGust': 40.27,
    'windGustTime': 1333162800,
    'windBearing': 302,
    'cloudCover': 0.71,
    'uvIndex': 3,
    'uvIndexTime': 1333191600,
    'visibility': 6.137,
    'temperatureMin': 38.68,
    'temperatureMinTime': 1333202400,
    'temperatureMax': 46.41,
    'temperatureMaxTime': 1333144800,
    'apparentTemperatureMin': 30.82,
    'apparentTemperatureMinTime': 1333202400,
    'apparentTemperatureMax': 40.05,
    'apparentTemperatureMaxTime': 1333144800}]},
     'offset': 2}